## Imports, Configs, and Constants

In [1]:
import polars as pl
import json
from web3 import Web3
from eth_abi import decode as decode_abi
from eth_account import Account
import math

mainnet = Web3(Web3.HTTPProvider("https://eth.llamarpc.com"))
anvil = Web3(Web3.HTTPProvider("http://localhost:8545"))

Account.enable_unaudited_hdwallet_features()
account = anvil.eth.account.from_mnemonic(
    "test test test test test test test test test test test junk"
)

default_txn = {
    'from': account.address,
    'gas': 2_000_000,
    'maxFeePerGas': anvil.to_wei(10, 'gwei'),
    'maxPriorityFeePerGas': anvil.to_wei(2, 'gwei'),
}

Uni v3 ETH/USDC 5bps Logs

In [2]:
df = pl.read_parquet("../../sample.parquet")

print(df.head())
print(df.columns)

shape: (5, 10)
┌────────────┬────────────┬───────────┬───────────┬───┬───────────┬───────────┬────────┬───────────┐
│ block_numb ┆ transactio ┆ log_index ┆ transacti ┆ … ┆ topic1    ┆ topic2    ┆ topic3 ┆ data      │
│ er         ┆ n_index    ┆ ---       ┆ on_hash   ┆   ┆ ---       ┆ ---       ┆ ---    ┆ ---       │
│ ---        ┆ ---        ┆ u32       ┆ ---       ┆   ┆ binary    ┆ binary    ┆ binary ┆ binary    │
│ u32        ┆ u32        ┆           ┆ binary    ┆   ┆           ┆           ┆        ┆           │
╞════════════╪════════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪════════╪═══════════╡
│ 17780001   ┆ 117        ┆ 411       ┆ [binary   ┆ … ┆ [binary   ┆ [binary   ┆ null   ┆ [binary   │
│            ┆            ┆           ┆ data]     ┆   ┆ data]     ┆ data]     ┆        ┆ data]     │
│ 17780006   ┆ 109        ┆ 311       ┆ [binary   ┆ … ┆ [binary   ┆ [binary   ┆ null   ┆ [binary   │
│            ┆            ┆           ┆ data]     ┆   ┆ data]     ┆ data]   

Get topic hashes for desired logs: Mint, Burn, and Swap

In [3]:
!cast sig-event "Mint(address sender, address indexed owner, int24 indexed tickLower, int24 indexed tickUpper, uint128 amount, uint256 amount0, uint256 amount1)"
!cast sig-event "Burn(address indexed owner, int24 indexed tickLower, int24 indexed tickUpper, uint128 amount, uint256 amount0, uint256 amount1)"
!cast sig-event "Swap(address indexed sender, address indexed recipient, int256 amount0, int256 amount1, uint160 sqrtPriceX96, uint128 liquidity, int24 tick)"

0x7a53080ba414158be7ec69b987b5fb7d07dee101fe85488f0853ae16239d0bde
0x0c396cd989a39f4459b5fa1aed6a9a8dcdbc45908acfd67e028cd568da98982c
0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67


In [4]:
mint_event_bytes = Web3.to_bytes(0x7a53080ba414158be7ec69b987b5fb7d07dee101fe85488f0853ae16239d0bde)
burn_event_bytes = Web3.to_bytes(0x0c396cd989a39f4459b5fa1aed6a9a8dcdbc45908acfd67e028cd568da98982c)
swap_event_bytes = Web3.to_bytes(0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67)

event_df = df.filter((pl.col("topic0") == mint_event_bytes) | (pl.col("topic0") == burn_event_bytes) | (pl.col("topic0") == swap_event_bytes))
print("All events: ", event_df.shape)
print("Mint events: ", event_df.filter(pl.col("topic0") == mint_event_bytes).shape)
print("Burn events: ", event_df.filter(pl.col("topic0") == burn_event_bytes).shape)
print("Swap events: ", event_df.filter(pl.col("topic0") == swap_event_bytes).shape)
event_df.head(8)

All events:  (246, 10)
Mint events:  (4, 10)
Burn events:  (4, 10)
Swap events:  (238, 10)


block_number,transaction_index,log_index,transaction_hash,contract_address,topic0,topic1,topic2,topic3,data
u32,u32,u32,binary,binary,binary,binary,binary,binary,binary
17780001,117,411,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]
17780006,109,311,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]
17780008,13,83,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]
17780009,71,171,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]
17780018,65,203,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]
17780021,1,14,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]
17780021,2,21,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]
17780022,157,361,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]


In [5]:
mint_decode_types = ["address", "uint128", "uint256", "uint256"]
burn_decode_types = ["uint128", "uint256", "uint256"]
swap_decode_types = ["int256", "int256", "uint160", "uint128", "int24"]

## Instantiate Contracts

In [6]:
with open("../../broadcast/Anvil.s.sol/31337/run-latest.json", "r") as f:
    deployment = json.load(f)

contract_creates = list(filter(lambda txn: txn['transactionType'] == 'CREATE', deployment['transactions']))

contract_names = [
    'MockERC20',
    'PoolManager',
    'PoolModifyPositionTest',
    'PoolSwapTest',
    'PoolDonateTest',
]

contracts = {}
for contract_name in contract_names:
    with open(f"../../out/{contract_name}.sol/{contract_name}.json", "r") as f:
        abi = json.load(f)['abi']
    
    if contract_name == 'MockERC20':
        # WETH
        address = list(
            filter(
                lambda txn: txn['contractName'] == contract_name and txn['arguments'][0] == 'Mock WETH',
                contract_creates
            )
        )[0]['contractAddress']
        print(f"{address}\tMock WETH")
        contracts['MWETH'] = anvil.eth.contract(address=address, abi=abi)

        # USDC
        address = list(
            filter(
                lambda txn: txn['contractName'] == contract_name and txn['arguments'][0] == 'Mock USDC',
                contract_creates
            )
        )[0]['contractAddress']
        print(f"{address}\tMock USDC")
        contracts['MUSDC'] = anvil.eth.contract(address=address, abi=abi)
    else:
        address = list(filter(lambda txn: txn['contractName'] == contract_name, contract_creates))[0]['contractAddress']
        print(f"{address}\t{contract_name}")
        contracts[contract_name] = anvil.eth.contract(address=address, abi=abi)


0x0165878A594ca255338adfa4d48449f69242Eb8F	Mock WETH
0xa513E6E4b8f2a923D98304ec87F64353C4D5C853	Mock USDC
0x5FbDB2315678afecb367f032d93F642f64180aa3	PoolManager
0xCf7Ed3AccA5a467e9e704C703E8D87F634fB0Fc9	PoolModifyPositionTest
0xDc64a140Aa3E981100a9becA4E685f962f0cF6C9	PoolSwapTest
0x5FC8d32690cc91D4c39d9d3abcBD16989F875707	PoolDonateTest


Define Pool Keys


In [7]:
currency0 = contracts['MUSDC'].address if anvil.to_int(hexstr=contracts['MUSDC'].address) < anvil.to_int(hexstr=contracts['MWETH'].address) else contracts['MWETH'].address
currency1 = contracts['MUSDC'].address if currency0 == contracts['MWETH'].address else contracts['MWETH'].address
print(currency0, currency1)

key0 = {
    "currency0": currency0,
    "currency1": currency1,
    "fee": 3000,
    "tickSpacing": 60,
    "hooks": "0x0000000000000000000000000000000000000000"
}

key1 = {
    "currency0": currency0,
    "currency1": currency1,
    "fee": 3000,
    "tickSpacing": 60,
    "hooks": "0x0c00000000000000000000000000000000000001"
}

0x0165878A594ca255338adfa4d48449f69242Eb8F 0xa513E6E4b8f2a923D98304ec87F64353C4D5C853


Initialize Pools

* Take the first `Mint()` event to get `sqrtPriceRatioX96`

In [8]:
mint_df = event_df.filter(pl.col("topic0") == mint_event_bytes)
(_, amount, amount0, amount1) = decode_abi(mint_decode_types, mint_df[0, 'data'])

tickLower = decode_abi(["int24"], mint_df[0, 'topic2'])
tickUpper = decode_abi(["int24"], mint_df[0, 'topic3'])

sqrtPriceX96 = math.floor(math.sqrt(amount1/amount0)*2**96)

# initialize pools
contracts['PoolManager'].functions['initialize'](
    key0, sqrtPriceX96
).transact(default_txn)

contracts['PoolManager'].functions['initialize'](
    key1, sqrtPriceX96
).transact(default_txn)


HexBytes('0xfeec73e6ff9076042b1a7285d0ffe0b33baeb98c6b841345aba8a825a17560cb')

## Simulation

In [9]:
def modifyLiquidity(poolKey, tickLower, tickUpper, liquidityDelta):
    positionParams = {
        "tickLower": tickLower,
        "tickUpper": tickUpper,
        "liquidityDelta": liquidityDelta,
    }
    contracts['PoolModifyPositionTest'].functions["modifyPosition"](
        poolKey,
        positionParams
    ).transact(default_txn)
    
def swap(poolKey, zeroForOne, amountSpecified):
    settings = {
        "withdrawTokens": True,
        "settleUsingTransfer": True
    }
    # parameters for infinite impact
    MIN_PRICE_LIMIT = 4295128739 + 1
    MAX_PRICE_LIMIT = 1461446703485210103287273052203988822378723970342 - 1
    params = {
        "zeroForOne": zeroForOne,
        "amountSpecified": amountSpecified,
        "sqrtPriceLimitX96": MIN_PRICE_LIMIT if zeroForOne else MAX_PRICE_LIMIT,
    }
    contracts['PoolSwapTest'].functions['swap'](
        poolKey,
        params,
        settings
    ).transact(default_txn)


In [24]:
# loop over the events and simulate the actions against the pools
for row in event_df.iter_rows(named=True):
    if row['topic0'] == mint_event_bytes:
        (sender, amount, amount0, amount1) = decode_abi(mint_decode_types, row['data'])
        owner = decode_abi(["address"], row['topic1'])
        tickLower = decode_abi(["int24"], row['topic2'])
        tickUpper = decode_abi(["int24"], row['topic3'])
        
    elif row['topic0'] == burn_event_bytes:
        (amount, amount0, amount1) = decode_abi(burn_decode_types, row['data'])
        owner = decode_abi(["address"], row['topic1'])
        tickLower = decode_abi(["int24"], row['topic2'])
        tickUpper = decode_abi(["int24"], row['topic3'])
    elif row['topic0'] == swap_event_bytes:
        (amount0, amount1, sqrtPriceX96, liquidity, tick) = decode_abi(swap_decode_types, row['data'])
        sender = decode_abi(["address"], row['topic1'])
        recipient = decode_abi(["address"], row['topic2'])
        # when amount1 < 0, the user traded token0 for token1
        zeroForOne = True if amount1 < 0 else False
        amountSpecified = amount0 if zeroForOne else amount1
        # swap(key0, zeroForOne=zeroForOne, amountSpecified=amountSpecified)